In [1]:
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder \
    .appName("Write to MySQL") \
    .config("spark.jars", "/Users/habaokhanh/.ivy2/jars/mysql-connector-java-8.0.30.jar")\
    .getOrCreate()

23/11/08 15:26:35 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
23/11/08 15:26:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/11/08 15:26:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
jdbc_url = "jdbc:mysql://localhost:3306"
db_name = "movie"
table_name = "log_movie_id"
mysql_url = f"{jdbc_url}/{db_name}"
db_properties = {
    "user": "root",
    "password": "h@b@0kh@nh", 
    "driver": "com.mysql.jdbc.Driver"
}

23/11/08 15:26:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
df = spark.read.json('/Users/habaokhanh/Study_BigData_Dataset/log_content/20220401.json')

In [8]:
df = df.select('_source.*')
df.show()

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|  KPLUS|HNH743103|402343C25D7D|          251|
|  KPLUS|HNH893773|B84DEE76D3B8|          924|
|  KPLUS|HND083642|B84DEE849A0F|         1444|
|  KPLUS|DNFD74404|90324BB44C39|          691|
|  KPLUS|DTFD21200|B84DEED27709|         1436|
|  KPLUS|LDFD05747|0C96E6C95E53|         1434|
|  KPLUS|HNH063566|B84DEEDD1C85|          687|
|  KPLUS|HNH866786|10394E2790A5|          248|
|  KPLUS|NBAAA1128|10394E47C1AF|          247|
|  KPLUS|HNH960439|B84DEED34371|          683|
|  KPLUS|HNJ035736|CCD4A1FA86A5|          246|
|  KPLUS|NTFD93673|B84DEEEF4763|         2288|
|  KPLUS|HNJ063267|10394E172CA7|         2282|
|  KPLUS|HNH790383|4CEBBD53378B|          906|
|  KPLUS|THFD

In [9]:
df.write\
    .mode("overwrite") \
    .jdbc(url=mysql_url, table=table_name, mode="overwrite", properties=db_properties)

In [10]:
df_read = spark.read \
    .jdbc(url=mysql_url, table="log_movie_id", properties=db_properties)
df_read.show()

+-------+---------+------------+-------------+
|AppName| Contract|         Mac|TotalDuration|
+-------+---------+------------+-------------+
|CHANNEL|STFD08699|08674EDF52FF|        16627|
|  KPLUS|HNH579912|0C96E62FC55C|          254|
|CHANNEL|DNH010243|C0B5D7E80B85|         1882|
|CHANNEL|HNH389848|5CEA1D708A36|         8940|
|CHANNEL|SGH547372|D89C67A91E45|           68|
|CHANNEL|HNH753478|B84DEE439A27|         8974|
|CHANNEL|TQAAA1190|B84DEED906A8|        16544|
|CHANNEL|SGH369727|E89EB419C993|           70|
|CHANNEL|TIFD23713|CCEDDC332157|         8940|
|  KPLUS|HUFD40665|CCEDDC333614|         1457|
|CHANNEL|BGFD41812|4CEBBD533B87|        16627|
|CHANNEL|SGH769363|E4AB89736830|           67|
|  KPLUS|HNH572635|B068E6A1C5F6|         2318|
|CHANNEL|DAFD90644|B84DEE78D4DD|         5591|
|CHANNEL|SGH387375|B84DEEF66F99|        16658|
|CHANNEL|HNH916275|18473D53DCD3|         8940|
|CHANNEL|DTD011928|B046FCACC100|         5591|
|  KPLUS|HND141717|08674EE8D2C2|         1452|
|CHANNEL|HNH0

In [11]:
spark.stop()

TRY ALL FUNCS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, desc
from pyspark.sql.functions import row_number
import pyspark.sql.functions as sf
from datetime import datetime
import os

In [2]:
def get_spark_session():
    # return SparkSession.builder.config("spark.driver.memory", "8g").config(
    #     "spark.executor.cores", 8).getOrCreate()
    return SparkSession.builder \
        .appName("Write to MySQL") \
        .config("spark.jars", "/Users/habaokhanh/.ivy2/jars/mysql-connector-java-8.0.30.jar")\
        .getOrCreate()

In [3]:
def read_json_data(spark, path, file_name):
    data = spark.read.json(path + file_name)
    return data.select('_source.*')

In [60]:
def add_type_column(df):
    tv_condition_list = ['CHANNEL', 'DSHD', 'KPLUS', 'KPlus']
    movie_condition_list = ['VOD', 'FIMS_RES', 'BHD_RES', 'VOD_RES', 'FIMS', 'BHD', 'DANET']
    df = df.withColumn("Type",
                       when(col("AppName").isin(*tv_condition_list), "Truyền Hình")
                       .when(col("AppName").isin(*movie_condition_list),"Phim Truyện")
                       .when(col("AppName") == 'RELAX', "Giải Trí")
                       .when(col("AppName") == 'CHILD', "Thiếu Nhi")
                       .when(col("AppName") == 'SPORT', "Thể Thao")
                       .otherwise("Error"))
    return df

def rename_column(df):
    df = df.withColumnRenamed('Giải Trí', 'RelaxDuration') \
        .withColumnRenamed('Phim Truyện', 'MovieDuration') \
        .withColumnRenamed('Thiếu Nhi', 'ChildDuration') \
        .withColumnRenamed('Thể Thao', 'SportDuration') \
        .withColumnRenamed('Truyền Hình', 'TVDuration')
    return df

In [43]:
def add_date_filter_non_error(df, file_date):
    df = df.withColumn('Date', sf.lit(file_date))
    df = df.filter(df.Type != 'Error').select('Contract', 'Type', 'TotalDuration', 'Date').dropDuplicates()
    return df

In [6]:
def calculate_most_watched(df):
    windowSpec = Window.partitionBy("Contract").orderBy(desc("TotalDuration"))
    most_watched = df.withColumn("rank", row_number().over(windowSpec))
    most_watched = most_watched.filter(most_watched.rank == 1)
    return most_watched.select("Contract", "Type").withColumnRenamed("Type", "MostWatch")

In [62]:
def calculate_customer_taste(df):
    for column in df.columns:
        if 'Duration' in column:
            df = df.withColumn(column, when(col(column).isNotNull(), column[:-8]))

    taste = df.withColumn('CustomerTaste',
                           sf.concat_ws("-", *[col(i) for i in df.columns if 'Duration' in i]))

    taste = taste.select('Contract', 'CustomerTaste')
    return taste

In [8]:
def calculate_activeness(df, start_date, end_date):
    total_days = (end_date - start_date).days + 1
    
    active = df.groupby('Contract', 'Date').agg((sf.sum('TotalDuration').alias('TotalDurationPerDay')))
    active = active.withColumn("IsActive", sf.when(active.TotalDurationPerDay > 0, 1).otherwise(0))

    activeness = active.groupBy("Contract").agg(sf.sum("IsActive").alias("ActiveDays"))
    activeness = activeness.withColumn("ActiveRate", (sf.col("ActiveDays") / sf.lit(total_days)))
    return activeness

In [9]:
def join_dataframes(final_df, most_watched_df, customer_taste_df, activeness_df):
    final_df = final_df.join(most_watched_df, on='Contract', how='left')
    final_df = final_df.join(customer_taste_df, on='Contract', how='left')
    final_df = final_df.join(activeness_df, on='Contract', how='left')
    return final_df

In [10]:
def write_data(df):
    # df.repartition(1).write.csv('/Users/habaokhanh/Study_BigData_Dataset/log_content/clean/df_clean1', header=True)
    jdbc_url = "jdbc:mysql://localhost:3306"
    db_name = "movie"
    table_name = "log_movie_olap"
    mysql_url = f"{jdbc_url}/{db_name}"
    db_properties = {
        "user": "root",
        "password": "h@b@0kh@nh", 
        "driver": "com.mysql.cj.jdbc.Driver"
    }
    df.write\
    .mode("overwrite") \
    .jdbc(url=mysql_url, table=table_name, mode="overwrite", properties=db_properties)

In [47]:
def main_task(start_date_str, end_date_str):
    start_time = datetime.now()

    # Extract
    path = '/Users/habaokhanh/Study_BigData_Dataset/log_content/'
    list_file = sorted([file for file in os.listdir(path) if file != '.DS_Store'])
    spark = get_spark_session()
    start_date = datetime.strptime(start_date_str, "%Y%m%d").date()
    end_date = datetime.strptime(end_date_str, "%Y%m%d").date()

    result_df = None
    for file_name in list_file:
        file_date_str = file_name.split('_')[-1].split('.')[0]
        file_date = datetime.strptime(file_date_str, "%Y%m%d").date()
        if start_date <= file_date <= end_date:
            df = read_json_data(spark, path, file_name)
            df = add_type_column(df)
            df = add_date_filter_non_error(df, file_date)

            if result_df is None:
                result_df = df
            else:
                result_df = result_df.union(df)
    
    return result_df


    # # Transform
    # result_df.show()
    # result_df.printSchema()

    # result_df_pivot = result_df.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
    # print(f'result_df_pivot is: {result_df_pivot.show()}')

    # most_watched_df = calculate_most_watched(result_df)
    # print(f'most_watched_df is: {most_watched_df.show()}')

    # customer_taste_df = calculate_customer_taste(result_df_pivot)
    # print(f'customer_taste_df is: {customer_taste_df.show()}')

    # activeness_df = calculate_activeness(result_df, start_date, end_date)
    # print(f'activeness_df is: {activeness_df.show()}')

    # # Join and print final dataframe
    # final_df = join_dataframes(result_df_pivot, most_watched_df, customer_taste_df, activeness_df)
    # print(f'final_df is: {final_df.show()}')

    # # Load
    # # write_data(final_df)
    # print('Data loaded to MySQL')

    # end_time = datetime.now()
    # print(f'It took {(end_time - start_time).total_seconds()} to process the data')

# RUN MAIN FUNCTION
# main_task('20220401', '20220402')

In [51]:
result_df = main_task('20220401', '20220402')

In [52]:
result_df.show()

+---------+-----------+-------------+----------+
| Contract|       Type|TotalDuration|      Date|
+---------+-----------+-------------+----------+
|BGFD29928|Truyền Hình|          669|2022-04-01|
|NDD011998|Truyền Hình|        18941|2022-04-01|
|HND486882|Phim Truyện|         5545|2022-04-01|
|QAFD05596|Truyền Hình|        83495|2022-04-01|
|HNH910024|Phim Truyện|         2847|2022-04-01|
|TGD020392|Phim Truyện|        17534|2022-04-01|
|TBFD09573|Phim Truyện|        14544|2022-04-01|
|TIFD29139|Phim Truyện|        25726|2022-04-01|
|BID025030|Truyền Hình|          170|2022-04-01|
|SGH069784|Truyền Hình|          151|2022-04-01|
|SGH936402|Truyền Hình|           61|2022-04-01|
|CBFD00028|Truyền Hình|        82141|2022-04-01|
|HPFD25462|Truyền Hình|        82775|2022-04-01|
|HND204270|Truyền Hình|        18770|2022-04-01|
|DAFD39835|Phim Truyện|         6728|2022-04-01|
|SGH661929|Phim Truyện|         7847|2022-04-01|
|BDAAA4088|Truyền Hình|          587|2022-04-01|
|HYFD17846|Truyền Hì

In [61]:
result_df_pivot = result_df.groupBy('Contract','Date').pivot("Type").sum('TotalDuration')
result_df_pivot = rename_column(result_df_pivot)
result_df_pivot.show()

23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:17:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+----------+-------------+-------------+-------------+-------------+----------+
| Contract|      Date|RelaxDuration|MovieDuration|ChildDuration|SportDuration|TVDuration|
+---------+----------+-------------+-------------+-------------+-------------+----------+
|KGFD30003|2022-04-01|         null|         null|         null|         null|      1243|
|DNFD42634|2022-04-01|         null|        12878|        12845|         null|      5226|
|SGH701193|2022-04-01|         null|         5549|         null|         null|       148|
|HUFD31379|2022-04-01|         null|         null|         null|         null|        41|
|SGJ036411|2022-04-01|         null|         null|         null|         null|        55|
|HNH952720|2022-04-01|         null|         null|         null|         null|        13|
|SGH536041|2022-04-01|         null|         null|         null|         null|      1458|
|DLD024779|2022-04-01|         null|         null|         null|         null|     11964|
|LCFD17923

In [54]:
most_watched_df = calculate_most_watched(result_df)
most_watched_df.show()

+--------------+-----------+
|      Contract|  MostWatch|
+--------------+-----------+
|113.182.209.48|   Giải Trí|
|     AGAAA0338|Truyền Hình|
|     AGAAA0342|Truyền Hình|
|     AGAAA0346|Truyền Hình|
|     AGAAA0353|Phim Truyện|
|     AGAAA0372|Truyền Hình|
|     AGAAA0391|Truyền Hình|
|     AGAAA0504|Truyền Hình|
|     AGAAA0544|Truyền Hình|
|     AGAAA0550|Truyền Hình|
|     AGAAA0555|Truyền Hình|
|     AGAAA0576|Truyền Hình|
|     AGAAA0613|Truyền Hình|
|     AGAAA0638|Truyền Hình|
|     AGAAA0663|Truyền Hình|
|     AGAAA0693|Truyền Hình|
|     AGAAA0718|Truyền Hình|
|     AGAAA0723|Truyền Hình|
|     AGAAA0732|Truyền Hình|
|     AGAAA0750|Truyền Hình|
+--------------+-----------+
only showing top 20 rows



In [57]:
def customer_taste_calc(df):
    df = df.withColumn("RelaxDuration",when(col("RelaxDuration").isNotNull(),"Relax").otherwise(col("RelaxDuration")))
    df = df.withColumn("MovieDuration",when(col("MovieDuration").isNotNull(),"Movie").otherwise(col("MovieDuration")))
    df = df.withColumn("ChildDuration",when(col("ChildDuration").isNotNull(),"Child").otherwise(col("ChildDuration")))
    df = df.withColumn("SportDuration",when(col("SportDuration").isNotNull(),"Sport").otherwise(col("SportDuration")))
    df = df.withColumn("TVDuration",when(col("TVDuration").isNotNull(),"TV").otherwise(col("TVDuration")))
    
    taste = df.withColumn('CustomerTaste', sf.concat_ws("-", *[i for i in df.columns if i != ['Contract']]))
    taste = taste.select('Contract','CustomerTaste')
    return taste

In [63]:
customer_taste_df = calculate_customer_taste(result_df_pivot)
customer_taste_df.show()

23/11/08 17:18:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:18:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:18:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:18:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:18:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/11/08 17:18:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+---------+--------------+
| Contract| CustomerTaste|
+---------+--------------+
|KGFD30003|            TV|
|DNFD42634|Movie-Child-TV|
|SGH701193|      Movie-TV|
|HUFD31379|            TV|
|SGJ036411|            TV|
|HNH952720|            TV|
|SGH536041|            TV|
|DLD024779|            TV|
|LCFD17923|            TV|
|BLFD04514|         Movie|
|QAFD07980|      Child-TV|
|SGH526551|            TV|
|DLFD08424|      Child-TV|
|QND032994|      Movie-TV|
|HNJ035836|            TV|
|BGFD37109|            TV|
|DAFD62269|            TV|
|SGFD96679|      Movie-TV|
|HDFD19918|            TV|
|CMFD16557|      Movie-TV|
+---------+--------------+
only showing top 20 rows

